# SPAM


In [1]:
import sqlite3
import requests  # To get the pages
import hashlib
import re

# from bs4 import BeautifulSoup # and to process them

con = sqlite3.connect("spam.db")
cur = con.cursor()


## Globals

In [2]:
base_url = "http://web.mit.edu/jync/www/spam/"
extension = ".html"

## Scraper Table

In [3]:
st = cur.execute("CREATE TABLE IF NOT EXISTS \"url_cache\" (\
	\"hash\"	TEXT,\
	\"text\"	TEXT\
);")


## Poem Table

In [4]:
# pt = cur.execute("DROP TABLE poems;")

pt = cur.execute("CREATE TABLE IF NOT EXISTS \"poems\" (\
	\"id\"	INTEGER UNIQUE,\
	\"line_1\"	TEXT,\
	\"line_2\"	TEXT,\
	\"line_3\"	TEXT,\
	\"author\"	TEXT\
);")
print(pt)

# Scraper

In [7]:
for i in range ( 1001,19601,100 ) :
    url = base_url + str(i) + "-" + str(i + 99) + extension
    # print(url)
    
    ID = i
    hash = hashlib.md5(url.encode()).hexdigest()
    
    # https://stackoverflow.com/a/9756276
    res = cur.execute("SELECT EXISTS(SELECT 1 FROM url_cache WHERE hash='"+hash+"');").fetchone()[0]
    
    if res == 0 :
        r = requests.get(url)
        text = r.text
        cur.execute("INSERT INTO url_cache VALUES ( ?, ?)", ( hash, text ) )
        # print(hash)

    else :
        # print(hash)
        continue 
        
con.commit()

In [8]:
for i in range ( 1001,19601,100 ) :
    url = base_url + str(i) + "-" + str(i + 99) + extension
    # print(url)
    
    ID = i
    hash = hashlib.md5(url.encode()).hexdigest()
    # print(hash)
    res = cur.execute("SELECT text FROM url_cache WHERE `hash` = '" + hash + "'" )
    text = res.fetchone()[0]
    
    # print(len(text))
    matches = re.findall(r"<SPAN .*\n.*\n.*\n.*\n.*\n.*\n.*<P>",text)
    
    for match in matches :
        # Found some matches
        # print( match )
        groups = re.findall(r"<SPAN CLASS='Number'>(.*)\.</SPAN><BR>\n.*\n(.*)<BR>\n(.*)<BR>\n(.*)<BR>\n.*\n.*>--(.*)</ADDRESS><P>", match)[0]
        
        if len( groups) == 5 :
            # https://stackoverflow.com/a/4869782

            poem_values = [ int(groups[0]), 
                           re.sub('<[^>]*>', '', groups[1]), 
                           re.sub('<[^>]*>', '', groups[2]),
                           re.sub('<[^>]*>', '', groups[3]),
                           re.sub('<[^>]*>', '', groups[4]),]
            # print(poem_values)

            insert_results = cur.execute("INSERT OR REPLACE INTO poems VALUES ( ?, ?, ?, ?, ? )", poem_values )
            # print(insert_results)
        else :
            print(len(groups))
    # break

con.commit()

In [157]:
con.close()

---

In [25]:
import pyphen
import syllapy
from collections import defaultdict

In [4]:
text = """
The two columns with Tags in their labels allow you to sort and search cases by each of several additional 
characteristics. 
To see more detailed definitions for some of these terms, hover over their full names or click on the links:



"""

In [5]:
dic = pyphen.Pyphen(lang='en')
dic.inserted(text)

'\nThe t-wo -columns with Tag-s in -their la-bels al-low y-ou to sort and search case-s by each of -sever-al ad-di-tion-al \nchar-ac-ter-ist-ic-s. \nTo see -more -de-tailed defin-i-tion-s -for -some of these term-s, hover- over- -their -full -names or click- on the link-s:\n\n\n\n'

In [8]:
count = syllapy.count('additional')

In [9]:
count

4

In [11]:
for word in text.split() :
    print(f"for the word {word}, we have {syllapy.count(word)} syllables.")

for the word The, we have 1 syllables.
for the word two, we have 1 syllables.
for the word columns, we have 2 syllables.
for the word with, we have 1 syllables.
for the word Tags, we have 1 syllables.
for the word in, we have 1 syllables.
for the word their, we have 1 syllables.
for the word labels, we have 2 syllables.
for the word allow, we have 2 syllables.
for the word you, we have 1 syllables.
for the word to, we have 1 syllables.
for the word sort, we have 1 syllables.
for the word and, we have 1 syllables.
for the word search, we have 1 syllables.
for the word cases, we have 2 syllables.
for the word by, we have 1 syllables.
for the word each, we have 1 syllables.
for the word of, we have 1 syllables.
for the word several, we have 3 syllables.
for the word additional, we have 4 syllables.
for the word characteristics., we have 5 syllables.
for the word To, we have 1 syllables.
for the word see, we have 1 syllables.
for the word more, we have 1 syllables.
for the word detailed, w

In [17]:
for word in text.split() :
    print('word')
    print(word)

word
word
word
word
word
word
word
word
word
word
word
word
word
word
word
word
word
word
word
word
word
word
word
word
word
word
word
word
word
word
word
word
word
word
word
word
word
word
word
word
word


In [29]:
haiku = """
An old silent pond
A frog jumps into the pond—
Splash! Silence again.
"""


haiku= """
A world of dew,
And within every dewdrop
A world of struggle.


"""

In [30]:
lines = haiku.split("\n")

In [31]:
lines = [line for line in lines if line]

In [32]:
lines

['A world of dew,', 'And within every dewdrop', 'A world of struggle.']

In [40]:
syllable_counts = defaultdict(int)

In [41]:
for line in lines :
    for word in line.split() :
        syllable_counts[line]+= syllapy.count(word)

In [43]:
#add tuples to show full analysis 
tuple(syllable_counts.values())

(4, 7, 5)

In [39]:
syllable_counts

defaultdict(int,
            {'A world of dew,': 4,
             'And within every dewdrop': 7,
             'A world of struggle.': 5})

In [ ]:
#look at patterns, potentially take 10K and analyze 